## Generating text for news headlines:
Language Modelling is the core problem for a number of of natural language processing tasks such as speech to text, conversational system, and text summarization. A trained language model learns the likelihood of occurrence of a word/ character based on the previous sequence of words/ characters used in the text. Language models can be operated at character level, n-gram level, sentence level or even paragraph level. We will create a language model for predicting next word by implementing and training state-of-the-art Recurrent Neural Networks under Deep Learning. 

We will use **Keras** library in python to implement our project.

### 1. Import the libraries

In [1]:
# Let us import required libraries first
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
import tensorflow.keras.utils as ku 
import tensorflow

# set seeds for reproducability
tensorflow.random.set_seed(200)

import pandas as pd
import numpy as np
import string

import warnings
warnings.filterwarnings("ignore")

### 2. Load the dataset

Load the dataset of news headlines

In [2]:
all_headlines = []
article_df = pd.read_csv('ArticlesMarch2018.csv')
all_headlines.extend(list(article_df.headline.values))
all_headlines = [h for h in all_headlines if h != "Unknown"]
len(all_headlines)

1250

In [3]:
all_headlines[:10]

['Virtual Coins, Real Resources',
 'U.S. Advances Military Plans for North Korea',
 'Mr. Trump and the ‘Very Bad Judge’',
 'To Erase Dissent, China Bans Pooh Bear and ‘N’',
 'Loans Flowed to Kushner Cos. After Visits to the White House',
 'China Envoy Intends To Ease Trade Tensions',
 'President Trump’s Contradictory, and Sometimes False, Comments About Gun Policy to Lawmakers',
 'Classic Letter Puzzle',
 'Silicon Valley Disruption In an Australian School',
 '‘The Assassination of Gianni Versace’ Episode 6: A Nothing Man']

### 3. Dataset preparation

#### 3.1 Dataset cleaning 

In dataset preparation step, we will first perform text cleaning of the data which includes removal of punctuations and lower casing all the words. 

In [4]:
def clean_text(txt):
    txt = "".join(v for v in txt if v not in string.punctuation).lower()
    txt = txt.encode("utf8").decode("ascii",'ignore')
    return txt 

corpus = [clean_text(x) for x in all_headlines]
corpus[:10]

['virtual coins real resources',
 'us advances military plans for north korea',
 'mr trump and the very bad judge',
 'to erase dissent china bans pooh bear and n',
 'loans flowed to kushner cos after visits to the white house',
 'china envoy intends to ease trade tensions',
 'president trumps contradictory and sometimes false comments about gun policy to lawmakers',
 'classic letter puzzle',
 'silicon valley disruption in an australian school',
 'the assassination of gianni versace episode 6 a nothing man']

#### 3.2 Generating Sequence of N-gram Tokens
The next step is Tokenization. Tokenization is a process of extracting tokens (terms / words) from a corpus. Python’s library Keras has inbuilt model for tokenization which can be used to obtain the tokens and their index in the corpus. After this step, every text document in the dataset is converted into sequence of tokens. 


In [5]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
token_list = tokenizer.texts_to_sequences(["I am happy to see you here today"])[0]
print(token_list)

check=[]

for i in range(1, len(token_list)):
  n_gram_sequence = token_list[:i+1]
  check.append(n_gram_sequence)

check

[33, 469, 1062, 3, 191, 16, 84, 767]


[[33, 469],
 [33, 469, 1062],
 [33, 469, 1062, 3],
 [33, 469, 1062, 3, 191],
 [33, 469, 1062, 3, 191, 16],
 [33, 469, 1062, 3, 191, 16, 84],
 [33, 469, 1062, 3, 191, 16, 84, 767]]

In [6]:
tokenizer = Tokenizer()

def get_sequence_of_tokens(corpus):
    ## tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1    
    
    ## convert data to sequence of tokens 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

inp_sequences, total_words = get_sequence_of_tokens(corpus)

inp_sequences[:10], total_words

([[1119, 1120],
  [1119, 1120, 116],
  [1119, 1120, 116, 1121],
  [31, 1122],
  [31, 1122, 589],
  [31, 1122, 589, 392],
  [31, 1122, 589, 392, 7],
  [31, 1122, 589, 392, 7, 61],
  [31, 1122, 589, 392, 7, 61, 70],
  [117, 10]],
 3582)

In the above output [1119, 1120], [1119, 1120,116], [1119, 1120, 116, 1121] and so on represents the ngram phrases generated from the input data. where every integer corresponds to the index of a particular word in the complete vocabulary of words present in the text. For example

**Headline:** i stand  with the shedevils  
**Ngrams:** | **Sequence of Tokens**

<table>
<tr><td>Ngram </td><td> Sequence of Tokens</td></tr>
<tr> <td>i stand </td><td> [30, 507] </td></tr>
<tr> <td>i stand with </td><td> [30, 507, 11] </td></tr>
<tr> <td>i stand with the </td><td> [30, 507, 11, 1] </td></tr>
<tr> <td>i stand with the shedevils </td><td> [30, 507, 11, 1, 975] </td></tr>
</table>



#### 3.3 Padding the Sequences and obtain Variables : Predictors and Target
Now we have generated a data-set which contains sequence of tokens. Before starting training the model, we need to pad the sequences and make their lengths equal. We can use pad_sequence function of Kears for this purpose. To input this data into a learning model, we need to create predictors and label. For example:


Headline:  they are learning data science

<table>
<tr><td>PREDICTORS </td> <td>           LABEL </td></tr>
<tr><td>they                   </td> <td>  are</td></tr>
<tr><td>they are               </td> <td>  learning</td></tr>
<tr><td>they are learning      </td> <td>  data</td></tr>
<tr><td>they are learning data </td> <td>  science</td></tr>
</table>

In [7]:
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)
predictors,label,len(label[0]),max_sequence_len

(array([[   0,    0,    0, ...,    0,    0, 1119],
        [   0,    0,    0, ...,    0, 1119, 1120],
        [   0,    0,    0, ..., 1119, 1120,  116],
        ...,
        [   0,    0,    0, ...,  979,  151,  386],
        [   0,    0,    0, ...,    0,    0, 3581],
        [   0,    0,    0, ...,    0, 3581,    5]]),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 3582,
 18)

Perfect, now we can obtain the input vector X (predictors) and the label vector Y (label) which can be used for the training purposes. Now we will create RNN model for out data.

### 4. RNN for Text Generation
Unlike Feed-forward neural networks in which activation outputs are propagated only in one direction, the activation outputs from neurons propagate in both directions (from inputs to outputs and from outputs to inputs) in Recurrent Neural Networks. This creates loops in the neural network architecture which acts as a ‘memory state’ of the neurons. This state allows the neurons an ability to remember what have been learned so far.

The memory state in RNNs gives an advantage over traditional neural networks. Lets architecture a RNN model in our code. We have added total three layers in the model.

1. Input Layer : Takes the sequence of words as input
2. RNN Layer : Computes the output using RNN units. We have added 200 units in the layer.
3. Dropout Layer : A regularisation layer which randomly turns-off the activations of some neurons in the RNN layer. It helps in preventing over fitting. 
4. Output Layer : Computes the probability of the best possible next word as output

We will run this model for total 100 epoochs but it can be experimented further.

In [8]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    
    # Setting early_stopping feature to stop early on getting stagnant
    early_stopping = EarlyStopping(
    min_delta=0.01, # minimium amount of change to count as an improvement
    patience=5, # how many epochs to wait before stopping
    restore_best_weights=True,
    )
    
    # Add Input Embedding Layer
    model.add(Embedding(total_words, 32, input_length=input_len))
   
    # input_dim: Integer. Size of the vocabulary, i.e. maximum integer index + 1.
    # output_dim: Integer. Dimension of the dense embedding.
    # input_length: Length of input sequences, when it is constant. 

    # Add Hidden Layer 1 - RNN Layer
    model.add(SimpleRNN(200))
    
    # Add Hidden Layer 2 - Dropout Layer
    model.add(Dropout(0.1))
    
    # Add Output Layer
    model.add(Dense(total_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    return model

model = create_model(max_sequence_len, total_words)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 17, 32)            114624    
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 200)               46600     
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense (Dense)                (None, 3582)              719982    
Total params: 881,206
Trainable params: 881,206
Non-trainable params: 0
_________________________________________________________________


Lets train our model now

In [9]:
model.fit(predictors, label, epochs=200)

Train on 8057 samples
Epoch 1/200
8057/8057 [==============================] - 10s 1ms/sample - loss: 7.6523
Epoch 2/200
8057/8057 [==============================] - 9s 1ms/sample - loss: 7.0929
Epoch 3/200
8057/8057 [==============================] - 9s 1ms/sample - loss: 6.7999
Epoch 4/200
8057/8057 [==============================] - 9s 1ms/sample - loss: 6.4093
Epoch 5/200
8057/8057 [==============================] - 10s 1ms/sample - loss: 5.9856
Epoch 6/200
8057/8057 [==============================] - 9s 1ms/sample - loss: 5.5373
Epoch 7/200
8057/8057 [==============================] - 11s 1ms/sample - loss: 5.0833
Epoch 8/200
8057/8057 [==============================] - 12s 1ms/sample - loss: 4.6307
Epoch 9/200
8057/8057 [==============================] - 10s 1ms/sample - loss: 4.1927
Epoch 10/200
8057/8057 [==============================] - 9s 1ms/sample - loss: 3.7565
Epoch 11/200
8057/8057 [==============================] - 9s 1ms/sample - loss: 3.3603
Epoch 12/200
8057/8057 [=

## 5. Generating the text 

Great, our model architecture is now ready and we can train it using our data. Next lets write the function to predict the next word based on the input words (or seed text). We will first tokenize the seed text, pad the sequences and pass into the trained model to get predicted word. The multiple predicted words can be appended together to get predicted sequence.


In [10]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()

## 6. Some Results

In [11]:
print (generate_text("jack ", 5, model, max_sequence_len))
print (generate_text("president trump",5, model, max_sequence_len))
print (generate_text("donald", 6, model, max_sequence_len))
print (generate_text("india and china", 2, model, max_sequence_len))
print (generate_text("new york", 4, model, max_sequence_len))
print (generate_text("science and technology", 6, model, max_sequence_len))

Jack  I Served In Vietnam Heres
President Trump Pushes Tariffs Latin America Links
Donald Trumps Tunnel Vision Killed Or In
India And China Envoy Intends
New York Has 7 Billion Reasons
Science And Technology Not Strictly Doctor Hollywood Night In
